In [73]:
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

### model training

In [74]:
train_data = []

with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)

In [78]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

<ipython-input-78-07b0c7f8f9f5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([entry["text"] for entry in train_data])


In [79]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
X_train_stats = np.array([np.mean(x) for x in X_train])
X_test_stats = np.array([np.mean(x) for x in X_test])

In [81]:
#initialize svm model
svm_model = SVC(C = 1.0, kernel='rbf',gamma='scale',degree=3, class_weight='balanced')

In [82]:
svm_model.fit(X_train_stats.reshape(-1, 1), y_train)

SVC(class_weight='balanced')

In [83]:
#prediction
y_pred = svm_model.predict(X_test_stats.reshape(-1, 1))

In [84]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5725641025641026


In [58]:
with open('../models/svm_model.pkl', 'wb') as model_file:
    pickle.dump(svm_model, model_file)

### Prediction

In [59]:
# 打开输出文件
with open('../data/Paul_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  # 写入CSV文件的标题行

    # 逐行读取测试数据集
    with open('../data/test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # 对特征进行统计（示例中使用平均值）
            X_test_stats = np.mean(text)

            # 使用模型进行预测
            prediction = svm_model.predict(np.array(X_test_stats).reshape(1, -1))

            # 将预测结果写入CSV文件
            output_file.write(f"{entry['id']},{prediction[0]}\n")